In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [ ]:
data = torch.load('data/test_dataset.pt')
l = len(data)

for i in range(0, len(data)):
    data[i][1] = [ data[i][1][63] / 2000 , data[i][1][64] / 2000 , data[i][1][65] / 2000 , data[i][1][66] / 2000 ]
    
    data[i][0][1:4] = np.true_divide( data[i][0][1:4], 30) # Normalize P1 top buildings
    data[i][0][4:7] = np.true_divide( data[i][0][4:7], 30) # Normalize P1 bottom buildings

    data[i][0][8:11] = np.true_divide( data[i][0][8:11], 30) # Normalize P2 top buildings
    data[i][0][11:14] = np.true_divide( data[i][0][11:14], 30) # Normalize P2 bottom buildings
    
    data[i][0][63] = data[i][0][63] / 2000 # Normalize P1 Top Nexus HP
    data[i][0][64] = data[i][0][64] / 2000 # Normalize P2 Top Nexus HP
    
    data[i][0][65] = data[i][0][65] / 2000 # Normalize P1 Bottom Nexus HP
    data[i][0][66] = data[i][0][66] / 2000 # Normalize P2 Bottom Nexus HP

    data[i][0][0] = data[i][0][10] / 1500 # Normalize P1 Minerals
    
    data[i][0] = data[i][0].reshape((2, 34))
#np.random.shuffle(data)

train_data = np.array(data[: int(np.floor(l * 0.8))])
test_data = np.array(data[int(np.floor(l * 0.8)) : ])
print(train_data.shape, test_data.shape)
print(data[0][0])
plt.imshow(data[0][0], interpolation='nearest')
plt.show()

In [ ]:
model = torch.load("./nexus-HP-transition-model-predict-4.pt")